In [7]:
#Importing Necessary Libraries
import os
import pandas as pd
import pickle
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Load Datasets
df = pd.read_csv("/Users/satwik/Documents/GitHub/cap5771sp25-project/Data/Cleaned Data/Modeling_Dataset_Enriched.csv")

# Create folder to save best models
model_dir = "/Users/satwik/Documents/GitHub/cap5771sp25-project/BestModels"
os.makedirs(model_dir, exist_ok=True)

# Intent Classification
df_intent = df[['Input', 'Intent']].dropna()
X = df_intent['Input']
y = df_intent['Intent']

vectorizer_intent = TfidfVectorizer(max_features=169)  # Matching feature size
X_vec = vectorizer_intent.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, stratify=y, random_state=42)

intent_model = RandomForestClassifier(n_estimators=100, class_weight='balanced')
intent_model.fit(X_train, y_train)

with open(f"{model_dir}/intent_classifier_random_forest.pkl", "wb") as f:
    pickle.dump(intent_model, f)

with open(f"{model_dir}/vectorizer_intent.pkl", "wb") as f:
    pickle.dump(vectorizer_intent, f)

# Gradio Chatbot App

In [9]:
import gradio as gr
import pickle
import os
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from sklearn.feature_extraction.text import TfidfVectorizer

# Load Models
model_dir = "/Users/satwik/Documents/GitHub/cap5771sp25-project/BestModels"

with open(os.path.join(model_dir, "intent_classifier_random_forest.pkl"), "rb") as f:
    intent_model = pickle.load(f)

with open(os.path.join(model_dir, "vectorizer_intent.pkl"), "rb") as f:
    intent_vectorizer = pickle.load(f)

# Load BlenderBot
blenderbot_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(blenderbot_name)
blenderbot = BlenderbotForConditionalGeneration.from_pretrained(blenderbot_name)

# Chat History
chat_history = []

# Response Logic
def chatbot_reply(user_message, history):
    global chat_history

    # Step 1: Intent Prediction
    X_vec = intent_vectorizer.transform([user_message])
    predicted_intent = intent_model.predict(X_vec)[0]

    # Step 2: Add User Message
    chat_history.append(("User", user_message))

    # Step 3: Build LLM Context (FIXED: Only recent user message passed)
    full_context = f"User: {user_message}"

    # Step 4: Generate LLM Response
    inputs = tokenizer(full_context, return_tensors="pt", truncation=True, max_length=128)
    reply_ids = blenderbot.generate(**inputs)
    response = tokenizer.decode(reply_ids[0], skip_special_tokens=True)

    # Step 5: Add Bot Response
    chat_history.append(("Bot", response))

    # Format for gr.Chatbot
    formatted_chat = []
    for speaker, message in chat_history:
        role = "user" if speaker == "User" else "assistant"
        formatted_chat.append({"role": role, "content": message})

    return formatted_chat, predicted_intent

# Clear Chat Function
def clear_chat():
    global chat_history
    chat_history.clear()
    return [], ""

# ==== Gradio UI ====
with gr.Blocks() as demo:
    gr.Markdown("### Mental Health Support Chatbot")
    gr.Markdown("AI-powered assistant trained on Real-Time Counseling and Support data. This chatbot Predicts your **Message Intent** and responds empathetically.")

    user_input = gr.Textbox(label="How are you feeling today?", placeholder="Type your thoughts here...", lines=2)

    with gr.Row():
        chatbot_ui = gr.Chatbot(label="Conversation", type="messages")
        intent_box = gr.Textbox(label="Predicted Intent", interactive=False)

    with gr.Row():
        submit_btn = gr.Button("Submit")
        clear_btn = gr.Button("Clear")

    submit_btn.click(fn=chatbot_reply, inputs=[user_input, chatbot_ui], outputs=[chatbot_ui, intent_box])
    clear_btn.click(fn=clear_chat, outputs=[chatbot_ui, intent_box])

# Launch with Shareable Public Link
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://25bb9bba52aaa9de9c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
